In [1]:
import torch
import requests
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

In [2]:
%matplotlib notebook
%matplotlib inline

OK_PATH='/Data_Storage/Rui_Data_Space/VQA/OK-VQA'
A_OK_PATH='/Data_Storage/Rui_Data_Space/VQA/A-OKVQA'
PATH='/Data_Storage/Rui_Data_Space/VQA'
import pandas as pd
from collections import defaultdict

In [3]:
import json
import pickle as pkl

def load_pkl(path):
    data=pkl.load(open(path,'rb'))
    return data

def load_json(path):
    data=json.load(open(path,'r'))
    return data

In [4]:
import os
import random

In [5]:
from sklearn.cluster import KMeans

In [6]:
from transformers import BertModel, AutoTokenizer
model_name='bert-base-uncased'
bert=BertModel.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
import openai
#engine_name='code-davinci-002'
engine_name='text-davinci-003'
temperature=0.5
max_tokens=128
top_p=1
frequency_penalty=0.0
presence_penalty=0.0
num_sequence=1
openai.api_key =open('../OK_VQA/openai_key.txt','r').readlines()[0].strip()

In [7]:
train_aokvqa=load_json(os.path.join(A_OK_PATH,'aokvqa_v1p0_train.json'))
val_aokvqa=load_json(os.path.join(A_OK_PATH,'aokvqa_v1p0_val.json'))
test_aokvqa=load_json(os.path.join(A_OK_PATH,'aokvqa_v1p0_test.json'))
print(len(train_aokvqa),len(val_aokvqa),len(test_aokvqa))

17056 1145 6702


In [8]:
captions=load_pkl('../EACL-24-results/aokvqa_val_captions_100.pkl')
print (len(captions))

1145


In [9]:
kb_info=load_pkl('../EACL-24-results/aokvqa_val_kb.pkl')

In [15]:
NUM_VIS=5
NUM_CAP=5
vis=0
#random.shuffle(vqa_all)
total_repre={}

for k,row in enumerate(val_aokvqa):
    
    ques_id=str(row['question_id'])
    if ques_id not in kb_info:
        continue
    kb=kb_info[ques_id]
    ques=row['question']
    cap=captions[ques_id]
    context=cap[:NUM_CAP]
    kb=kb_info[ques_id]
    
    all_sent=[]
    all_sent.append('Question:'+ques)
    all_sent.append('Context:'+' . '.join(context)+' . ')
    all_sent.append('Knowledge:'+kb)
    all_sent='\n'.join(all_sent)
    #print (all_sent)
    tokens=tokenizer(all_sent, padding='longest', 
                     truncation=True, max_length=512,
                     return_tensors="pt").input_ids
    repre=bert(tokens)[0][:,0].detach().squeeze().numpy()#768,
    total_repre[ques_id]=repre
    #print(repre.shape)
    vis+=1

In [16]:
pkl.dump(total_repre,open('../EACL-24-results/a-okvqa-repre.pkl','wb'))

In [10]:
total_repre=load_pkl('../EACL-24-results/a-okvqa-repre.pkl')

In [11]:
print (len(total_repre))

1145


In [12]:
from sklearn.cluster import KMeans

matrix=[]
for name in total_repre.keys():
    matrix.append(total_repre[name])
print (len(matrix))
names=list(total_repre.keys())

NUM_CLUSTERS=8
n_clusters = NUM_CLUSTERS
kmeans = KMeans(n_clusters=n_clusters,
                init='k-means++',
                random_state=42)
kmeans.fit(matrix)
labels = kmeans.labels_

1145


In [13]:
ques_id_label_mapper={}
for i,name in enumerate(names):
    ques_id_label_mapper[name]=labels[i]
    
print (len(ques_id_label_mapper))

1145


In [14]:
clustered_row={i:[] for i in range(NUM_CLUSTERS)}
for k,row in enumerate(val_aokvqa):
    ques_id=str(row['question_id'])
    if ques_id not in kb_info:
        continue
    cluster_id=ques_id_label_mapper[ques_id]
    clustered_row[cluster_id].append(row)
for idx in range(NUM_CLUSTERS):
    print (idx, '\t',len(clustered_row[idx]))

0 	 230
1 	 172
2 	 161
3 	 174
4 	 33
5 	 113
6 	 123
7 	 139


In [15]:
NUM_CAP=3
NUM_CLUSTERS=8
def generate_context(row):
    ques_id=str(row['question_id'])
    cap=captions[ques_id]
    context=cap[:NUM_CAP]
    ques=row['question']
    kb=kb_info[ques_id]
    text=[]
    text.append('Context:'+' . '.join(context))
    text.append('Question:'+ques)
    text.append('Knowledge:'+kb)
    text='\n'.join(text)
    return text

In [16]:
def generate(label):
    prompt_head='Please generate related background knowledge to the question:'
    all_row=[]
    all_row.append(prompt_head)
    for idx in range(NUM_CLUSTERS):
        if idx == label:
            continue
        cur_cluster=clustered_row[idx]
        select_id=random.randint(0,len(cur_cluster)-1)
        all_row.append(generate_context(cur_cluster[select_id]))
    all_row='\n\n'.join(all_row)
    return all_row

In [17]:
import time

In [ ]:
#need to do some visualization
NUM_QUERY_TIMES=20
vis=0
#time.sleep(60)
invalid=defaultdict(int)
all_names=[]
for k,row in enumerate(val_aokvqa):
    """if vis>1500:
        break"""
    if k%100==0:
        print ('Already finished:',
               k,k*100.0/len(val_aokvqa),
               '\n\tText rather than code:',
               len(all_names),
               len(os.listdir('../A_OKVQA/cluster_generated_kb')) )
    #if vis>0:
    #    break
    ques_id=str(row['question_id'])

    if os.path.exists('../A_OKVQA/cluster_generated_kb/'+ques_id+'.json'):
        all_pieces=load_json('../A_OKVQA/cluster_generated_kb/'+ques_id+'.json')
        if len(all_pieces)>=NUM_QUERY_TIMES:
            continue
    else:
        all_pieces=[]
    if ques_id not in kb_info:
        label=random.randint(0,7)
    else:
        label=ques_id_label_mapper[ques_id]
    all_names.append(ques_id)
    #all_pieces=[]
    
    cap=captions[ques_id]
    context=cap[:NUM_CAP]
    ques=row['question']
    #print(ques,'\n',context)
    if vis<3:
        print (vis)
        print(ques,'\n',context)
    for _ in range(NUM_QUERY_TIMES-len(all_pieces)):
        demonstration=generate(label)
        inputs=[demonstration]
        inputs.append('\nContext:'+'. '.join(context))
        inputs.append('Question:'+ques)
        inputs='\n'.join(inputs)+'\nKnowledge:'
        try:
            response = openai.Completion.create(engine=engine_name,
                                                prompt=inputs,
                                                temperature=temperature,
                                                max_tokens=max_tokens,
                                                top_p=top_p,
                                                frequency_penalty=frequency_penalty,
                                                presence_penalty=presence_penalty,
                                                stop=["\n"])
            knowledge=response["choices"][0]["text"].strip()
            all_pieces.append(knowledge)
            if vis<3:
                print ('\t',knowledge)
            time.sleep(0.5)
        except:
            invalid[ques_id]+=1
            #openai.api_key =open('OK_VQA/openai_key.txt','r').readlines()[0].strip()
            print ('Invalid id:',ques_id)
            #time.sleep(0.5)
    #print ('\n')
    json.dump(all_pieces,
              open(
                  '../A_OKVQA/cluster_generated_kb/'+ques_id+'.json',
                  'w'))
    
    if vis<3:
        print ('\n')
    vis+=1